In [ ]:
import pandas as pd
import numpy as np
import torch  # Chuyển từ TF sang Torch
import os
from sklearn.metrics import confusion_matrix
import json
import time
from tqdm import tqdm
from metrics import compute_metrics
from data_preparation import prepare_csv_datasets

# Đường dẫn tới file model SPLAM (thường là splam_script.pt)
MODEL_PATH = r"D:\Bio_sequence_Research_AITALAB\SPLAM\models\splam_script.pt"
PREPARED_FOLDER = "prepared_data/"
RESULTS_DIR = "results/"
BATCH_SIZE = 256  
CONTEXT = 400     
TARGET_LEN = 800
test_files = ['test_1_1_1.csv', 'test_2_1_1.csv', 'test_4_1_1.csv', 'test_10_1_1.csv', 'test_data.csv']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Data preparation

In [5]:
os.makedirs(PREPARED_FOLDER, exist_ok=True)
prepare_csv_datasets(test_files)

[00:57:43] Loading Genome with pyfaidx...
Sorting test_1_1_1.csv for sequential disk access...
🚀 Processing: test_1_1_1.csv (26310 rows)


Extracting: 100%|██████████| 26310/26310 [00:01<00:00, 24702.66it/s]



Type       | Window around center (-2 to +2) | Found?
-----------------------------------------------------------------
Donor      | AA[GG]TT                  | ❌ (Target: GT)
Donor      | CA[GG]TA                  | ❌ (Target: GT)
Donor      | CA[AG]TA                  | ❌ (Target: GT)
Donor      | AA[GG]TA                  | ❌ (Target: GT)
Donor      | AT[GG]TG                  | ❌ (Target: GT)
Acceptor   | AT[AG]GT                  | ✅ (Target: AG)
Acceptor   | CC[AG]AG                  | ✅ (Target: AG)
Acceptor   | GA[AG]GT                  | ✅ (Target: AG)
Acceptor   | GC[AG]GT                  | ✅ (Target: AG)
Acceptor   | AG[AG]CC                  | ✅ (Target: AG)
✅ Saved to prepared_data/test_1_1_1.csv | Speed: 8493.61 seq/s

Sorting test_2_1_1.csv for sequential disk access...
🚀 Processing: test_2_1_1.csv (35132 rows)


Extracting: 100%|██████████| 35132/35132 [00:01<00:00, 24192.44it/s]



Type       | Window around center (-2 to +2) | Found?
-----------------------------------------------------------------
Donor      | GG[TA]AT                  | ❌ (Target: GT)
Donor      | GA[GG]TG                  | ❌ (Target: GT)
Donor      | GG[TA]AA                  | ❌ (Target: GT)
Donor      | AA[GG]TG                  | ❌ (Target: GT)
Donor      | GG[TA]AG                  | ❌ (Target: GT)
Acceptor   | CC[AG]TG                  | ✅ (Target: AG)
Acceptor   | GC[AG]CC                  | ✅ (Target: AG)
Acceptor   | TC[AG]AA                  | ✅ (Target: AG)
Acceptor   | TC[AG]GC                  | ✅ (Target: AG)
Acceptor   | TT[AG]GA                  | ✅ (Target: AG)
✅ Saved to prepared_data/test_2_1_1.csv | Speed: 8355.16 seq/s

Sorting test_4_1_1.csv for sequential disk access...
🚀 Processing: test_4_1_1.csv (52776 rows)


...


Type       | Window around center (-2 to +2) | Found?
-----------------------------------------------------------------
Donor      | GG[TA]AG                  | ❌ (Target: GT)
Donor      | CC[GG]TG                  | ❌ (Target: GT)
Donor      | GG[TC]AG                  | ❌ (Target: GT)
Donor      | GG[TG]AG                  | ❌ (Target: GT)
Donor      | GA[GG]TA                  | ❌ (Target: GT)
Acceptor   | TC[AG]GT                  | ✅ (Target: AG)
Acceptor   | AG[AG]AA                  | ✅ (Target: AG)
Acceptor   | AC[AG]TT                  | ✅ (Target: AG)
Acceptor   | GC[AG]GA                  | ✅ (Target: AG)
Acceptor   | AG[GT]GG                  | ❌ (Target: AG)
✅ Saved to prepared_data/test_4_1_1.csv | Speed: 8139.52 seq/s

Sorting test_10_1_1.csv for sequential disk access...
🚀 Processing: test_10_1_1.csv (105708 rows)


Extracting: 100%|██████████| 105708/105708 [00:04<00:00, 24718.31it/s]



Type       | Window around center (-2 to +2) | Found?
-----------------------------------------------------------------
Donor      | GG[TT]AG                  | ❌ (Target: GT)
Donor      | CA[GG]TA                  | ❌ (Target: GT)
Donor      | AC[GG]TG                  | ❌ (Target: GT)
Donor      | GG[TC]AG                  | ❌ (Target: GT)
Donor      | AA[GG]TA                  | ❌ (Target: GT)
Acceptor   | AG[CT]GC                  | ❌ (Target: AG)
Acceptor   | GC[AG]TA                  | ✅ (Target: AG)
Acceptor   | AG[GT]GG                  | ❌ (Target: AG)
Acceptor   | AG[AG]GG                  | ✅ (Target: AG)
Acceptor   | CC[AG]CA                  | ✅ (Target: AG)
✅ Saved to prepared_data/test_10_1_1.csv | Speed: 8423.27 seq/s

Sorting test_data.csv for sequential disk access...
🚀 Processing: test_data.csv (938297 rows)


Extracting: 100%|██████████| 938297/938297 [00:37<00:00, 25041.36it/s]



Type       | Window around center (-2 to +2) | Found?
-----------------------------------------------------------------
Donor      | GG[TG]AG                  | ❌ (Target: GT)
Donor      | GG[TG]AG                  | ❌ (Target: GT)
Donor      | GG[TA]AG                  | ❌ (Target: GT)
Donor      | GA[GG]TG                  | ❌ (Target: GT)
Donor      | AA[GG]TG                  | ❌ (Target: GT)
Acceptor   | AG[CT]GT                  | ❌ (Target: AG)
Acceptor   | CT[AG]GC                  | ✅ (Target: AG)
Acceptor   | CT[AG]TG                  | ✅ (Target: AG)
Acceptor   | AG[GG]CG                  | ❌ (Target: AG)
Acceptor   | CC[AG]GG                  | ✅ (Target: AG)
✅ Saved to prepared_data/test_data.csv | Speed: 8451.58 seq/s



In [12]:
# df = pd.read_csv(r'D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceAI\prepared_data\test_1_1_1.csv')
# # Kiểm tra 5 mẫu Donor đầu tiên
# donor_samples = df[df['Splicing_types'] == 1]['sequence'].values[:5]
# for i, seq in enumerate(donor_samples):
#     # Lấy vùng trung tâm (index 5000 là nucleotide tại vị trí pos)
#     center = seq[5000:5002] 
#     print(f"Donor {i}: Vị trí trung tâm là '{center}' (Kỳ vọng thường là GT)")

# # Kiểm tra 5 mẫu Acceptor đầu tiên
# acceptor_samples = df[df['Splicing_types'] == 2]['sequence'].values[:5]
# for i, seq in enumerate(acceptor_samples):
#     center = seq[5000:5002]
#     print(f"Acceptor {i}: Vị trí trung tâm là '{center}' (Kỳ vọng thường là AG)")

# def diagnose_splice_sites(df, sample_size=5):
#     print(f"{'Type':<10} | {'Window (-5 to +5 around center)':<20} | {'Found?'}")
#     print("-" * 50)
    
#     for label, name in [(1, 'Donor'), (2, 'Acceptor')]:
#         samples = df[df['Splicing_types'] == label].sample(min(sample_size, len(df)))
#         for _, row in samples.iterrows():
#             seq = row['sequence']
#             # Lấy 11 nucleotide (vị trí 5000 là trung tâm)
#             window = seq[4995:5006]
            
#             target = "GT" if label == 1 else "AG"
#             found = "✅" if target in window else "❌"
            
#             # Highlight vị trí trung tâm bằng dấu ngoặc []
#             display_win = window[:5] + "[" + window[5:7] + "]" + window[7:]
#             print(f"{name:<10} | {display_win:<20} | {found} (Target: {target})")

# # Chạy sau khi load df từ prepared_data
# diagnose_splice_sites(df)

### Inference

In [ ]:
# Load model SPLAM (TorchScript)
print(f"Loading SPLAM Model on {device}...")
model = torch.jit.load(MODEL_PATH)
model.to(device)
model.eval()
print("Successfully loaded SPLAM model.")

In [ ]:
os.makedirs(RESULTS_DIR, exist_ok=True)

def one_hot_encode_splam(seq):
    """
    Encoding cho SPLAM: Shape (4, 800)
    Thứ tự base: A, C, G, T
    """
    seq_code = np.zeros((4, len(seq)), dtype=np.float32)
    seq = seq.upper()
    sn = np.frombuffer(seq.encode('ascii'), dtype=np.int8)
    
    seq_code[0, sn == ord('A')] = 1.0
    seq_code[1, sn == ord('C')] = 1.0
    seq_code[2, sn == ord('G')] = 1.0
    seq_code[3, sn == ord('T')] = 1.0
    return seq_code

def get_batches(sequences, batch_size):
    for i in range(0, len(sequences), batch_size):
        batch_seqs = sequences[i:i + batch_size]
        # Encode và stack thành tensor (Batch, 4, 800)
        batch_data = np.array([one_hot_encode_splam(s) for s in batch_seqs])
        yield torch.tensor(batch_data, dtype=torch.float32).to(device)

# --- CHƯƠNG TRÌNH CHÍNH ---
test_files = ['test_data.csv'] # Cập nhật danh sách file của bạn

pbar_files = tqdm(test_files, desc="Overall Progress")

for csv_file in pbar_files:
    path = os.path.join(PREPARED_FOLDER, csv_file)
    if not os.path.exists(path): continue
    
    df = pd.read_csv(path)
    y_true = df['Splicing_types'].values
    sequences = df['sequence'].values
    
    all_probs = []
    inference_start = time.time()
    
    with torch.no_grad():
        for batch_tensor in tqdm(get_batches(sequences, BATCH_SIZE), 
                                total=int(np.ceil(len(sequences)/BATCH_SIZE)), 
                                desc=" ↪ Inferencing", leave=False):
            
            # SPLAM Forward Pass
            # Output shape thường là (Batch, 3, 800) hoặc (Batch, 3) tùy version
            outputs = model(batch_tensor)
            
            # SPLAM thường dự đoán cho mọi vị trí hoặc vị trí trung tâm
            # Nếu output là (Batch, 3, 800), ta lấy tại index 400 (vị trí junction)
            if outputs.dim() == 3:
                # Lấy xác suất tại điểm giữa
                # SPLAM mapping: 0: None, 1: Acceptor, 2: Donor
                probs = outputs[:, :, 400].cpu().numpy()
            else:
                probs = outputs.cpu().numpy()
                
            all_probs.append(probs)

    avg_probs = np.vstack(all_probs)
    total_duration = time.time() - inference_start
    
    # --- MAPPING NHÃN THEO YÊU CẦU ---
    # SPLAM Gốc: 0=Null, 1=Acceptor, 2=Donor
    # Yêu cầu của bạn: 0=Null, 1=Donor, 2=Acceptor
    
    p_null = avg_probs[:, 0]
    p_acceptor_raw = avg_probs[:, 1] 
    p_donor_raw = avg_probs[:, 2]
    
    # Tạo mảng xác suất đã map: [p_null, p_donor, p_acceptor]
    probs_mapped = np.stack([p_null, p_donor_raw, p_acceptor_raw], axis=1)
    
    # Quyết định nhãn (Argmax hoặc Threshold)
    preds_mapped = np.argmax(probs_mapped, axis=1)
    
    # (Tùy chọn) Áp dụng Threshold nếu cần độ nhạy cao
    # THRESHOLD = 0.1
    # for i in range(len(preds_mapped)):
    #     if p_donor_raw[i] > THRESHOLD or p_acceptor_raw[i] > THRESHOLD:
    #         preds_mapped[i] = 1 if p_donor_raw[i] > p_acceptor_raw[i] else 2
    #     else: preds_mapped[i] = 0

    # --- LƯU KẾT QUẢ ---
    try:
        # Giả sử hàm compute_metrics của bạn nhận (y_true, y_pred, probs)
        # kết quả trả về một dict
        from metrics import compute_metrics 
        results = compute_metrics(y_true, preds_mapped, probs=probs_mapped)
        
        results['benchmarking'] = {
            'total_inference_time_sec': round(total_duration, 4),
            'samples_per_second': round(len(df) / total_duration, 2)
        }
        
        cm = confusion_matrix(y_true, preds_mapped)
        output = {"metrics": results, "confusion_matrix": cm.tolist()}
        
        json_path = os.path.join(RESULTS_DIR, csv_file.replace('.csv', '_results.json'))
        with open(json_path, 'w') as f:
            json.dump(output, f, indent=4)
            
        print(f"✅ {csv_file} processed. Speed: {results['benchmarking']['samples_per_second']} seq/s")
            
    except Exception as e:
        print(f"\n❌ Error with {csv_file}: {e}")

print("\n✨ SPLAM INFERENCE COMPLETED!")

Overall Progress:   0%|          | 0/5 [00:00<?, ?file/s, file=test_1_1_1.csv]

--- ĐANG KIỂM TRA ONE-HOT ENCODING ---
✅ HÀNG 14218: Khớp hoàn toàn (Độ dài: 10001)
✅ HÀNG 18004: Khớp hoàn toàn (Độ dài: 10001)
✅ HÀNG 15326: Khớp hoàn toàn (Độ dài: 10001)


206/206 [==============================] - 65s 306ms/step


206/206 [==============================] - 62s 302ms/step


206/206 [==============================] - 62s 301ms/step


...

7331/7331 [==============================] - 2187s 298ms/step


7331/7331 [==============================] - 2061s 281ms/step



--- Kiểm tra phân phối xác suất cho test_data.csv ---
   Nhãn 1: Avg = 0.382683, Max = 0.999861
   Nhãn 2: Avg = 0.322296, Max = 0.999870


Overall Progress: 100%|██████████| 5/5 [3:41:42<00:00, 2660.57s/file, file=test_data.csv]

✅ test_data.csv | Enc: 2115.72s | Total: 10740.92s

✨ ALL DATASETS PROCESSED SUCCESSFULLY!
